In [1]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
from dateutil import parser
from dateutil.relativedelta import relativedelta
from collections import Counter
import math

# Open and Pre-process

In [2]:
data_location = 'data'
# outlooks, pph, reports = read_datasets(data_location)
outlooks = gp.read_file(data_location + '/outlooks/all_outlooks_1.shp')
outlooks = outlooks.append(gp.read_file(data_location + '/outlooks/all_outlooks_2.shp'))
pph = xr.open_dataset(data_location + '/pph/all_pph.nc')

C:\Users\miles\AppData\Local\Temp\ipykernel_35864\1347856224.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outlooks = outlooks.append(gp.read_file(data_location + '/outlooks/all_outlooks_2.shp'))


In [3]:
tz_conversions = {'PST': timedelta(hours=8),
                  'MST': timedelta(hours=7),
                  'CST': timedelta(hours=6),
                  'CSt': timedelta(hours=6),
                  'CSC': timedelta(hours=6),
                  'SCT': timedelta(hours=6),
                  'EST': timedelta(hours=5),
                  'ESt': timedelta(hours=5),
                  'PDT': timedelta(hours=7),
                  'MDT': timedelta(hours=6),
                  'CDT': timedelta(hours=5),
                  'EDT': timedelta(hours=4),
                  'HST': timedelta(hours=10),
                  'SST': timedelta(hours=11),
                  'GST': timedelta(hours=10),
                  'AKS': timedelta(hours=9),
                  'AST': timedelta(hours=4),
                  'UNK': timedelta(hours=5),
                  'GMT': timedelta(0)}

def get_reports_date_strings(date_times, timezones):
    # returns list of strings of date of given datetime and timezone (where day cutoffs are 12-12 UTC) formatted as 'YYYYMMDD0000'
    for datetime, timezone, i in zip(date_times, timezones, range(len(timezones))):
        #print(datetime + ' ' + timezone[:3])
        datetime = parser.parse(datetime)
        datetime = datetime + tz_conversions[timezone[:3]]
        #print(datetime)
        if (datetime.hour < 12):
            datetime = datetime - timedelta(days = 1)
        if datetime.year > 2049:
            datetime = datetime - relativedelta(years = 100)
        datetime = datetime.strftime("%Y%m%d") + '0000'
        if i == 0:
            ret = [datetime]
        else:
            ret.append(datetime)
    return ret

def get_pph_date_strings(times):
    # returns a list of strings of given dates formatted as 'YYYYMMDD0000'
    for datetime, i in zip(times, range(len(times))):
        string = datetime.dt.strftime("%Y%m%d").values + '0000'
        if i == 0:
            ret = [string]
        else:
            ret.append(string)
    return ret

In [4]:
#reports['DATE'] = get_reports_date_strings(reports['BEGIN_DATE_TIME'], reports['CZ_TIMEZONE']) 
pph['time'] = get_pph_date_strings(pph.time) 
# subset outlooks into only one day 1, two day 2, and one day 3 categorical outlooks 
# day 3: cycle not -1. day 2: cycle not -1. Day 1: cycle 6. Category: categorical. 
#outlooks = outlooks[(((outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6)) | ((outlooks['DAY'] == 2) & (outlooks['CYCLE'] != -1)) | ((outlooks['DAY'] == 3) & (outlooks['CYCLE'] != -1)))
#        & (outlooks['CATEGORY'] == 'CATEGORICAL')]

# reset incicies
outlooks = outlooks.reset_index(drop=True)
#reports = reports.drop(columns=['geometry'])

In [5]:
old_outlook_dataset = xr.open_dataset('data/outlooks/grid_outlooks.nc')

# Gridize outlooks

In [6]:
def get_outlooks_subset(outlooks, outlook_type):
    outlooks = outlooks[(outlooks['THRESHOLD'] != 'SIGN') & (outlooks['THRESHOLD'] != 'TSTM')]
    if outlook_type == 'Day 3':
        return outlooks[(outlooks['DAY'] == 3) & (outlooks['CYCLE'] != -1) & (outlooks['CATEGORY'] == 'ANY SEVERE')]
    elif outlook_type == 'Day 2 7':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 7) & (outlooks['CATEGORY'] == 'ANY SEVERE')]
    elif outlook_type == 'Day 2 17':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 17) & (outlooks['CATEGORY'] == 'ANY SEVERE')]
    elif outlook_type == 'Day 1 Wind':
        return outlooks[(outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6) & (outlooks['CATEGORY'] == 'WIND')]
    elif outlook_type == 'Day 1 Hail':
        return outlooks[(outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6) & (outlooks['CATEGORY'] == 'HAIL')]
    elif outlook_type == 'Day 1 Tornado':
        return outlooks[(outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6) & (outlooks['CATEGORY'] == 'TORNADO')]
    
    elif outlook_type == 'Day 2 7 Wind':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 7) & (outlooks['CATEGORY'] == 'WIND')]
    elif outlook_type == 'Day 2 17 Wind':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 17) & (outlooks['CATEGORY'] == 'WIND')]
    elif outlook_type == 'Day 2 7 Hail':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 7) & (outlooks['CATEGORY'] == 'HAIL')]
    elif outlook_type == 'Day 2 17 Hail':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 17) & (outlooks['CATEGORY'] == 'HAIL')]
    elif outlook_type == 'Day 2 7 Tornado':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 7) & (outlooks['CATEGORY'] == 'TORNADO')]
    elif outlook_type == 'Day 2 17 Tornado':
        return outlooks[(outlooks['DAY'] == 2) & (outlooks['CYCLE'] == 17) & (outlooks['CATEGORY'] == 'TORNADO')]
    
    raise Exception("Invalid outlook_type given to get_outlooks_subset")

In [8]:
# as in mca: create_gridded_outlook_dataset. But will need to work overall, not just by hazard. day 3 is overall probability, days 1 and 2 by hazard
# day 3 use ANY SEVERE, day 2 use ANY SEVERE, day 1 construct from each hazard (assuming independence? or use highest prob, which basically assumes complete dependence? Is this what is done for day 2?)
# Things are entirely categorical before 2002, so even more reason to make that the cutoff.

outlook_types = ['Day 3', 'Day 2 7', 'Day 2 17', 
                 'Day 2 7 Wind', 'Day 2 7 Hail', 'Day 2 7 Tornado', 
                 'Day 2 17 Wind', 'Day 2 17 Hail', 'Day 2 17 Tornado', 
                 'Day 1 Wind', 'Day 1 Hail', 'Day 1 Tornado', 
                 'Day 1']

outlook_dataset = xr.Dataset(
    data_vars=dict(
        lat=(['y', 'x'], pph['lat'].data),
        lon=(['y', 'x'], pph['lon'].data)
    ),
    coords=dict(
        time=(['time'], pph['time'].data),
        x=(['x'], pph['x'].data),
        y=(['y'], pph['y'].data),
        outlook=(['outlook'], outlook_types)
    ),
    attrs=dict(description="outlook as a percentage as a function of date, lat/lon, and which hazard type",
            grid = pph.grid),
)

outlook_dataset = outlook_dataset.assign(prob = (('time', 'y', 'x', 'outlook'), np.full((len(outlook_dataset['time']), len(outlook_dataset['y']), len(outlook_dataset['x']), len(outlook_types)), 0.0)))





In [8]:
# adding an additional year
outlook_dataset = old_outlook_dataset.reindex(time=pph['time'], fill_value=0)
del old_outlook_dataset
outlook_dataset
# just full of zeros in new year, fill below

<xarray.Dataset>
Dimensions:  (time: 16436, x: 93, y: 65, outlook: 13)
Coordinates:
  * time     (time) <U12 '197901010000' '197901020000' ... '202312310000'
  * x        (x) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 88.0 89.0 90.0 91.0 92.0
  * y        (y) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 60.0 61.0 62.0 63.0 64.0
  * outlook  (outlook) object 'Day 3' 'Day 2 7' ... 'Day 1 Tornado' 'Day 1'
Data variables:
    lat      (y, x) float64 ...
    lon      (y, x) float64 ...
    prob     (time, y, x, outlook) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    description:  outlook as a percentage as a function of date, lat/lon, and...
    grid:         80-km NCEP 211

In [12]:
todo_times = outlook_dataset['time'][outlook_dataset['time'].str.startswith('2023')]

In [13]:
save_location = 'data/outlooks/grid_outlooks2.nc'

In [ ]:
# for each outlook type
for outlook_type in ['Day 3', 'Day 2 7', 'Day 2 17', 'Day 2 7 Wind', 'Day 2 7 Hail', 'Day 2 7 Tornado', 'Day 2 17 Wind', 'Day 2 17 Hail', 'Day 2 17 Tornado']:
    print(outlook_type)
    outlooks_subset = get_outlooks_subset(outlooks, outlook_type)
    oldyear = 0
    array = np.zeros((len(todo_times.values), len(outlook_dataset.y.values), len(outlook_dataset.x.values)))

    for date, i in zip(todo_times.values, range(len(todo_times.values))):
        lats = outlook_dataset.lat
        lons = outlook_dataset.lon
        datestring = date
        
        # printing to track progress
        year = datestring[0:4]
        if year != oldyear:
            print(year)
            oldyear = year

        outlooks_date = outlooks_subset[outlooks_subset['DATE'] == datestring]
        if (len(outlooks_date) > 0):
            for x, j in zip(outlook_dataset.x.values, range(len(outlook_dataset.x.values))):
                for y, k in zip(outlook_dataset.y.values, range(len(outlook_dataset.y.values))):
                    lat = lats.sel(x = x, y = y).values
                    lon = lons.sel(x = x, y = y).values
                    array[i, k, j] = find_threshold(outlooks_date, lat, lon)
    me = xr.DataArray(array, coords={'time': outlook_dataset.time.values, 'y': outlook_dataset.y.values, 'x': outlook_dataset.x.values},
                      dims=['time', 'y', 'x'])
    outlook_dataset['prob'].loc[dict(outlook = outlook_type)] = me
    outlook_dataset.to_netcdf(save_location)

Day 3
2023


In [142]:
outlook_dataset.to_netcdf(save_location)

In [ ]:
outlook_dataset['prob'].mean()

In [ ]:
outlook_dataset = xr.open_dataset(save_location)
outlook_dataset

In [ ]:
# build day 1 and Feb 1 2020- day 2 probabilities: max (assuming total dependence) seems right, set to True
dependent = True

for outlook in ['Day 2 7', 'Day 2 17', 'Day 1']:
    print(outlook)
    year = ''
    for time in outlook_dataset['time']:
        time = str(time.values)
        newyear = time[0:4]
        if newyear != year:
            print(newyear)
            year = newyear
        if dependent:
            m = m = outlook_dataset['prob'].sel(time = time, outlook = [outlook + ' Wind', outlook + ' Hail', outlook + ' Tornado']).max(dim = 'outlook')
        else:
            m = (1-(1-outlook_dataset['prob'].sel(time = time, outlook = [outlook + ' Wind'])).data * (1-outlook_dataset['prob'].sel(time = time, outlook = [outlook + ' Hail'])).data * (1-outlook_dataset['prob'].sel(time = time, outlook = [outlook + ' Tornado'])).data)[:, :, 0]
        if m.max() > 0:
            outlook_dataset['prob'].loc[dict(time = time, outlook = outlook)] = m

# Plot

In [ ]:
# check that max was a reasonable way to combine hazard probabilities
plt.plot(np.convolve(outlook_dataset['prob'].sel(outlook = 'Day 1').max(dim=['x', 'y']).data[8489:], np.ones(365)/365, 'same'))
plt.plot(np.convolve(outlook_dataset['prob'].sel(outlook = 'Day 2 17').max(dim=['x', 'y']).data[8489:], np.ones(365)/365, 'same'))
plt.plot(np.convolve(outlook_dataset['prob'].sel(outlook = 'Day 2 7').max(dim=['x', 'y']).data[8489:], np.ones(365)/365, 'same'))
plt.plot(np.convolve(outlook_dataset['prob'].sel(outlook = 'Day 3').max(dim=['x', 'y']).data[8489:], np.ones(365)/365, 'same'))
plt.legend(['Day 1', 'Day 2 17z', 'Day 2 7z', 'Day 3'])
if dependent:
    plt.title('Daily Maximum Any-Hazard Probability Over Time')
else:
    plt.title('Daily Maximum Any-Hazard Probability Over Time (assuming independent hazards)')
plt.ylabel('1-year Running Mean Daily Maximum Any-Hazard Probability')
plt.xlabel('Days Since March 30, 2002')
if dependent:
    plt.savefig('plots/prob_over_time.png')
else:
    plt.savefig('plots/prob_over_time_indep.png')


# This is kind of an interesting result on its own. Forecast practices have changed... higher risks issued less liberally